In [1]:
%matplotlib widget

import logging

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from loess.loess_1d import loess_1d

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas.coverages import CoverageInternal

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
cov = CoverageInternal(configuration=db.get_coverage_configuration_by_name(session, "tas_barometro_climatico"), identifier="tas_barometro_climatico-rcp26")

In [3]:
df = operations.get_climate_barometer_time_series(settings, cov)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 124 entries, NaT to NaT
Empty DataFrame


In [5]:
df.columns

Index([], dtype='object')

In [6]:
opendap_url = "/".join((
    settings.thredds_server.base_url, 
    settings.thredds_server.opendap_service_url_fragment, 
    cov.configuration.get_thredds_url_fragment(cov.identifier)
))

In [7]:
opendap_url

'http://thredds:8080/thredds/dodsC/ensymbc/std/clipped/fldmean/tas_avg_rcp26_ts19762100_ls_VFVG_fldmean.nc'

In [8]:
ds = netCDF4.Dataset(opendap_url)

In [22]:
cftime.num2pydate(
    ds.variables["time"][:], 
    units=ds.variables["time"].units, 
    calendar=ds.variables["time"].calendar, 
    #only_use_cftime_datetimes=False, 
    # only_use_python_datetimes=True
).to_datetimeindex()

AttributeError: 'MaskedArray' object has no attribute 'to_datetimeindex'

In [11]:
pd.DataFrame(
    pd.Series(
        cftime.num2date(
            ds.variables["time"][:],
            units=ds.variables["time"].units,
            calendar=ds.variables["time"].calendar,
            only_use_cftime_datetimes=False,
            only_use_python_datetimes=True),
        name="time"
    ),
    pd.Series(
        ds.variables[cov.configuration.netcdf_main_dataset_name][:].ravel(), 
        name=cov.identifier
    )
)

,time
tas_barometro_climatico-rcp26,
10.947353,NaT
10.956053,NaT
11.216602,NaT
11.034557,NaT
10.855161,NaT
...,...
12.250976,NaT
12.219064,NaT
12.196585,NaT


In [9]:
import cftime

In [24]:
pd.Series(ds.variables[cov.configuration.netcdf_main_dataset_name][:], name=cov.identifier)

ValueError: Data must be 1-dimensional